<a href="https://colab.research.google.com/github/fatimakgeneng/LangChain-Function-Tool-Calling-Calculator/blob/main/LangChain_Function_Tool_Calling_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%pip install -q -U python-dotenv langchain google-generativeai

In [20]:
import os
from dotenv import load_dotenv
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [21]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use Python's eval to compute the result safely
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

In [22]:
from langchain.tools import tool

# Define the tool using a decorator
@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.agents import initialize_agent, Tool, AgentType, AgentExecutor

# Initialize the Gemini model
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", api_key=os.environ["GOOGLE_API_KEY"])

# Define tools
tools = [calculator]

# Create a ToolAgent for the calculator
agent = initialize_agent(tools, gemini_model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)
executor = AgentExecutor.from_agent_and_tools(agent, tools)


In [55]:
from langchain.memory import ConversationBufferMemory

# Set up memory to maintain conversation context
memory = ConversationBufferMemory()

# Build the conversational chain
chain = ConversationChain(
    llm=gemini_model,
    memory=memory
)

In [56]:
# User query
query = "What is 15 divided by 3?"

# Get response from the system
response = chain.run(query)
print(response)

15 divided by 3 is 5.  That's a simple division problem;  15 contains three groups of five, or five groups of three.  It's a fundamental arithmetic operation that forms the basis of many more complex calculations.  I can perform much more complex mathematical operations as well, if you'd like!  Just let me know.



In [57]:
queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", chain.run(q))
    print("-" * 40)

Query: What is 25 multiplied by 4?
Response: 25 multiplied by 4 is 100.  That's equivalent to adding four 25s together (25 + 25 + 25 + 25 = 100).  You could also think of it as 25 doubled (50) and then doubled again (100).  Multiplication is the inverse operation of division, and understanding both is crucial for grasping many mathematical concepts.  This calculation is also easily visualized – imagine four rows of 25 objects each; counting them all would yield 100.  I can handle more intricate multiplication problems too, or any other mathematical operation you might have in mind.

----------------------------------------
Query: Now divide the result by 5.
Response: 100 divided by 5 is 20.  This is because five goes into 100 twenty times (5 x 20 = 100).  You can also think of it as dividing 100 into five equal groups, with each group containing 20.  This again demonstrates the relationship between multiplication and division.  As you can see, I can easily handle these kinds of sequent